# AOS_DM-29811  update ROTANG in ts_wep test files 

I think these files need to be re-created , since they were made with the bug in `comcam/segmentation.txt`... 

Test their header content:

In [9]:
from lsst.ts.wep.Utility import getModulePath, runProgram
from astropy.io import fits
import os 

testDir = os.path.join(getModulePath(), "tests")

imgFiles = os.path.join(
    getModulePath(),
    "tests",
    "testData",
    "phosimOutput",
    "realComCam",
    "repackagedFiles",
    "intra",
    "MC_H_20211231_006002_R22_S10.fits",
)

hdul = fits.open(imgFiles)

for i in range(1,len(hdul)):
    a = [f'{key}: {hdul[i].header[key]}' for key in  ['AMPID','GAIN','BIAS', 'ROTANG']]
    print(a)

['AMPID: C10', 'GAIN: 1.11531', 'BIAS: 1000.8', 'ROTANG: 0.0']
['AMPID: C11', 'GAIN: 1.12227', 'BIAS: 1044.94', 'ROTANG: 0.0']
['AMPID: C12', 'GAIN: 1.14053', 'BIAS: 1031.69', 'ROTANG: 0.0']
['AMPID: C13', 'GAIN: 1.16278', 'BIAS: 1036.65', 'ROTANG: 0.0']
['AMPID: C14', 'GAIN: 1.16361', 'BIAS: 1030.36', 'ROTANG: 0.0']
['AMPID: C15', 'GAIN: 1.18769', 'BIAS: 992.125', 'ROTANG: 0.0']
['AMPID: C16', 'GAIN: 1.16909', 'BIAS: 983.392', 'ROTANG: 0.0']
['AMPID: C17', 'GAIN: 1.15185', 'BIAS: 986.993', 'ROTANG: 0.0']
['AMPID: C07', 'GAIN: 1.12377', 'BIAS: 973.994', 'ROTANG: 0.0']
['AMPID: C06', 'GAIN: 1.12231', 'BIAS: 975.66', 'ROTANG: 0.0']
['AMPID: C05', 'GAIN: 1.12907', 'BIAS: 997.593', 'ROTANG: 0.0']
['AMPID: C04', 'GAIN: 1.13058', 'BIAS: 1047.91', 'ROTANG: 0.0']
['AMPID: C03', 'GAIN: 1.13623', 'BIAS: 970.172', 'ROTANG: 0.0']
['AMPID: C02', 'GAIN: 1.13295', 'BIAS: 968.795', 'ROTANG: 0.0']
['AMPID: C01', 'GAIN: 1.15074', 'BIAS: 960.55', 'ROTANG: 0.0']
['AMPID: C00', 'GAIN: 1.14892', 'BIAS: 1010

In [3]:
# compare to the content of obs_lsst .... 

# show the content of the obs lsst mapper 
# should be exactly the same as above ... 
from lsst.obs.lsst import LsstCam
camera = LsstCam().getCamera()
detector = camera['R22_S10']
for amp in detector:
    print(amp.getName(), amp.getGain())

C10 1.11531
C11 1.12227
C12 1.14053
C13 1.16278
C14 1.16361
C15 1.18769
C16 1.16909
C17 1.15185
C07 1.12377
C06 1.12231
C05 1.12907
C04 1.13058
C03 1.13623
C02 1.13295
C01 1.15074
C00 1.14892


ok - so these images had exactly the same gain, since they were made with `lsst/segmentation.txt`, i.e. 
its R22 of LsstCam,  rather than R22 of LsstComCam ... 

How does the `CamDataCollector` ingest images? 




In [4]:
from lsst.ts.wep.CamDataCollector import CamDataCollector
import tempfile
dataDir = tempfile.TemporaryDirectory(dir=testDir)
isrDir = tempfile.TemporaryDirectory(dir=dataDir.name)

camDataCollector = CamDataCollector(isrDir.name)

by using `gen2`  ingestImages.py , where only `lsstCam` is allowed. So I couldn't regenerate here lsstComCam data, because they would need extra work to make `obs_lsst` ingest `lsstComCam` e-images... So wait till `ts_wep` is entirely in gen3. 

## Update just the `ROTANG` header 

In [17]:
positions = ['extra','intra']
sensors  = ['S11','S10']

ampFiles = []
eImgFiles = []
for position in positions:
    for sensor in sensors:
        if position == 'extra':
            obsid = '1'
        elif position == 'intra':
            obsid = '2'
            
            
        fnameRoot =  os.path.join(
                                getModulePath(),
                                "tests",
                                "testData",
                                "phosimOutput",
                                "realComCam",
                                "repackagedFiles",
                                position,
                                f"MC_H_20211231_00600{obsid}_R22_{sensor}"
                                )
        
        ampFile = fnameRoot+".fits"           
        eFile = fnameRoot+"_e.fits.gz"
                              
        print(ampFile)
        print(eFile)
        ampFiles.append(ampFile)
        eImgFiles.append(eFile)
        


/project/scichris/aos/ts_wep/tests/testData/phosimOutput/realComCam/repackagedFiles/extra/MC_H_20211231_006001_R22_S11.fits
/project/scichris/aos/ts_wep/tests/testData/phosimOutput/realComCam/repackagedFiles/extra/MC_H_20211231_006001_R22_S11_e.fits.gz
/project/scichris/aos/ts_wep/tests/testData/phosimOutput/realComCam/repackagedFiles/extra/MC_H_20211231_006001_R22_S10.fits
/project/scichris/aos/ts_wep/tests/testData/phosimOutput/realComCam/repackagedFiles/extra/MC_H_20211231_006001_R22_S10_e.fits.gz
/project/scichris/aos/ts_wep/tests/testData/phosimOutput/realComCam/repackagedFiles/intra/MC_H_20211231_006002_R22_S11.fits
/project/scichris/aos/ts_wep/tests/testData/phosimOutput/realComCam/repackagedFiles/intra/MC_H_20211231_006002_R22_S11_e.fits.gz
/project/scichris/aos/ts_wep/tests/testData/phosimOutput/realComCam/repackagedFiles/intra/MC_H_20211231_006002_R22_S10.fits
/project/scichris/aos/ts_wep/tests/testData/phosimOutput/realComCam/repackagedFiles/intra/MC_H_20211231_006002_R22_S1

In [13]:
ampFiles

['/project/scichris/aos/ts_wep/tests/testData/phosimOutput/realComCam/repackagedFiles/extra/MC_H_20211231_006001_R22_S11.fits',
 '/project/scichris/aos/ts_wep/tests/testData/phosimOutput/realComCam/repackagedFiles/extra/MC_H_20211231_006001_R22_S10.fits',
 '/project/scichris/aos/ts_wep/tests/testData/phosimOutput/realComCam/repackagedFiles/intra/MC_H_20211231_006002_R22_S11.fits',
 '/project/scichris/aos/ts_wep/tests/testData/phosimOutput/realComCam/repackagedFiles/intra/MC_H_20211231_006002_R22_S10.fits']

In [5]:
os.listdir(os.path.join(
    getModulePath(),
    "tests",
    "testData",
    "phosimOutput",
    "realComCam",
    "repackagedFiles",
    "extra"))

['._.DS_Store',
 'MC_H_20211231_006001_R22_S11_e.fits.gz',
 'MC_H_20211231_006001_R22_S10.fits',
 '.DS_Store',
 'MC_H_20211231_006001_R22_S11.fits',
 'MC_H_20211231_006001_R22_S10_e.fits.gz']

In [12]:
# Update the header for amp images:



In [14]:
for fname in ampFiles:
    print(f'Updating {fname} header')
    
    # open the file 
    hdul = fits.open(fname)
    for i in range(1,len(hdul)):
        # open the header 
        header  = hdul[i].header
        print(header['AMPID'], header['ROTANG'])
        header['ROTANG'] += 90. 
        hdul[i].header =  header 
        print('-->', header['AMPID'], header['ROTANG'])
    hdul.writeto(fname, overwrite=True)

Updating /project/scichris/aos/ts_wep/tests/testData/phosimOutput/realComCam/repackagedFiles/extra/MC_H_20211231_006001_R22_S11.fits header
C10 0.0
--> C10 90.0
C11 0.0
--> C11 90.0
C12 0.0
--> C12 90.0
C13 0.0
--> C13 90.0
C14 0.0
--> C14 90.0
C15 0.0
--> C15 90.0
C16 0.0
--> C16 90.0
C17 0.0
--> C17 90.0
C07 0.0
--> C07 90.0
C06 0.0
--> C06 90.0
C05 0.0
--> C05 90.0
C04 0.0
--> C04 90.0
C03 0.0
--> C03 90.0
C02 0.0
--> C02 90.0
C01 0.0
--> C01 90.0
C00 0.0
--> C00 90.0
Updating /project/scichris/aos/ts_wep/tests/testData/phosimOutput/realComCam/repackagedFiles/extra/MC_H_20211231_006001_R22_S10.fits header
C10 0.0
--> C10 90.0
C11 0.0
--> C11 90.0
C12 0.0
--> C12 90.0
C13 0.0
--> C13 90.0
C14 0.0
--> C14 90.0
C15 0.0
--> C15 90.0
C16 0.0
--> C16 90.0
C17 0.0
--> C17 90.0
C07 0.0
--> C07 90.0
C06 0.0
--> C06 90.0
C05 0.0
--> C05 90.0
C04 0.0
--> C04 90.0
C03 0.0
--> C03 90.0
C02 0.0
--> C02 90.0
C01 0.0
--> C01 90.0
C00 0.0
--> C00 90.0
Updating /project/scichris/aos/ts_wep/tests/test

In [18]:
# Update the e-image header too 

for fname in eImgFiles:
    print(f'\nUpdating {fname} header')
    
    # open the file 
    hdul = fits.open(fname)
    print('before: ', hdul[0].header['ROTANG'])
    hdul[0].header['ROTANG'] += 90
    print('after: ', hdul[0].header['ROTANG'])
    hdul.writeto(fname, overwrite=True)
    
   


Updating /project/scichris/aos/ts_wep/tests/testData/phosimOutput/realComCam/repackagedFiles/extra/MC_H_20211231_006001_R22_S11_e.fits.gz header
before:  0.0
after:  90.0

Updating /project/scichris/aos/ts_wep/tests/testData/phosimOutput/realComCam/repackagedFiles/extra/MC_H_20211231_006001_R22_S10_e.fits.gz header
before:  0.0
after:  90.0

Updating /project/scichris/aos/ts_wep/tests/testData/phosimOutput/realComCam/repackagedFiles/intra/MC_H_20211231_006002_R22_S11_e.fits.gz header
before:  0.0
after:  90.0

Updating /project/scichris/aos/ts_wep/tests/testData/phosimOutput/realComCam/repackagedFiles/intra/MC_H_20211231_006002_R22_S10_e.fits.gz header
before:  0.0
after:  90.0


In [43]:
hdul[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
EXPTIME =                 15.0                                                  
DARKTIME=                 15.0                                                  
RUNNUM  = '9006001 '                                                            
MJD-OBS =     59579.9999131944                                                  
DATE-OBS= '2021-12-31T23:59:52.500'                                             
DAYOBS  = '20211231'                                                            
DATE    = '2021-03-04T05:27:13'                                                 
MJD     =    59277.22723379629                                                  
FILTER  = 'g       '        

## Update version history

.. _lsst.ts.wep-1.6.0:

-------------
1.6.0
-------------

* Update ROTANG header in realComcam test files




## Update tags 


git checkout develop
git tag -ln
git pull
git tag -a v1.6.2 

Merge pull request #81 from lsst-ts/tickets/DM-29811
Update ROTANG header in test files



